# Flappy Bird Junco Sim
2D flight sim for EECE5500 Mobile Robotics

In [1]:
############################### Import libraries ###############################
import os
import glob
import time
from datetime import datetime

import matplotlib.pyplot as plt

import numpy as np
from numpy import matlib

import gym

In [146]:
class Drone():
    m = 3.2
    Iyy = 1/12*m*0.8**2

    S = 0.25
    c = 0.13

    Cl_0 = 0.5
    Cl_alpha = 0.17
    
    Cd_0 = 0.02
    K = 0.01

    Cm_0 = -0.05
    Cm_alpha = 0.2
    Cm_alpha_dot = -0.1
    Cm_delta_e = 0.5

    def __init__(self):
        self.x_thres = 1000
        self.z_thresHigh = 500
        self.z_thresLow = 0
        self.v_thres = 50
        self.theta_thres = np.pi/6
        self.theta_dot_thres = np.pi
        self.gamma_thres = np.pi/6

        self.limits = np.array([
            self.x_thres,
            self.z_thresHigh,
            self.z_thresLow,
            self.v_thres,
            self.theta_thres,
            self.theta_dot_thres,
            self.gamma_thres
        ])

        self.state = None
        self.prev_action = None
        self.steps_beyond_terminated = None

        self.time = 0.
        self.dt = 0.02

    def lift(self,rho):
        vel = self.state[2]
        alpha = self.state[3] - self.state[4]

        Cl = self.Cl_0 + self.Cl_alpha*alpha

        return 0.5*rho*vel**2*self.S*Cl
    
    def drag(self,rho):
        vel = self.state[2]
        alpha = self.state[3] - self.state[4]

        Cl = self.Cl_0 + self.Cl_alpha*alpha
        Cd = self.Cd_0 + self.K*Cl**2

        return 0.5*rho*vel**2*self.S*Cd
    
    def moment(self,alpha_dot,delta_e,rho):
        vel = self.state[2]
        alpha = self.state[3] - self.state[4]
        Cm = self.Cm_0 + self.Cm_alpha*alpha + self.Cm_alpha_dot*alpha_dot + self.Cm_delta_e*delta_e

        return 0.5*rho*vel**2*self.S*self.c*Cm
    
    def step(self, action):
        err_msg = f"{action!r} ({type(action)}) invalid"
        assert self.state is not None, "Call reset before using step method."
        
        g = 9.81
        m = self.m
        self.time += self.dt
        rho = np.random.normal(1.225,0.01)

        L = self.lift(rho)

        if L <= m*g and self.state[1]<=0.1:
            x, z, v, theta, theta_dot, gamma = np.array([self.state[0],0,self.state[2],0,0,0])
        else:
            x, z, v, theta, theta_dot, gamma = self.state
        thrust, delta_e = action

        D = self.drag(rho)

        alpha = theta - gamma
        v_dot = (-D - m*g*np.sin(gamma) + thrust*np.cos(alpha)) / m
        gamma_dot = (L - m*g*np.cos(gamma) -thrust*np.sin(alpha)) / (m*v)
        x_dot = v*np.cos(gamma)
        z_dot = v*np.sin(gamma)

        alpha_dot = theta_dot - gamma_dot
        M = self.moment(alpha_dot,delta_e,rho)
        theta_ddot = M / self.Iyy

        

        # integrate
        x += x_dot*self.dt
        z += z_dot*self.dt
        v += v_dot*self.dt
        theta += theta_dot*self.dt #Removed second order terms.  We don't do that here (Also needs to be 1/2dt^2 f"(t))
        theta_dot += theta_ddot * self.dt
        gamma += gamma_dot*self.dt

        print(v_dot, gamma_dot,theta_dot)

        state = np.array([x,z,v,theta,theta_dot,gamma])
        action = np.array([thrust,delta_e])

        self.state = (list(np.reshape(state,(6,))))

        terminated = bool(
            x < -self.x_thres
            or x > self.x_thres
            or z > self.z_thresHigh
            or z < self.z_thresLow
            or v < -self.v_thres
            or v > self.v_thres
            or theta < -self.theta_thres
            or theta > self.theta_thres
            or theta_dot < -self.theta_dot_thres
            or theta_dot > self.theta_dot_thres
            or gamma < -self.gamma_thres
            or gamma > self.gamma_thres
        )

        if not terminated:
            reward = 1.0
        elif self.steps_beyond_terminated is None:
            self.steps_beyond_terminated = 0
            reward = 0.0
        else:
            if self.steps_beyond_terminated == 0:
                gym.logger.warn(
                    "You are calling 'step()' even though this "
                    "environment has already returned terminated = True. You "
                    "should always call 'reset()' once you receive 'terminated = "
                    "True' -- any further steps are undefined behavior."
                )
            self.steps_beyond_terminated += 1
            reward = 0.0

        return state, reward, terminated, False

    def reset(self):
        self.state = np.array([0,100,20,0,0,0])
        self.steps_beyond_terminated = None
        self.time = 0

        return self.state

    def close(self):
        pass

In [161]:
env = Drone()
state = env.reset()
action = np.array([1.45,0.1055])
fig, ax = plt.subplots(1,figsize=(10,5))
trajx = []
trajz = []

for t in range(1, 500):
    state, reward, done, _ = env.step(action)
    x = state[0]
    z = state[1]
    v = state[2]
    theta = -state[3]
    theta_dot = -state[4]
    gamma = state[5]
    trajx.append(x)
    trajz.append(z)

ax.plot(trajx, trajz)
fig.show()

0.024260421014267244 -0.01398380112696368 0.0012559341034872936
0.028380034898831813 -0.015676417534627785 0.0020021515842158794
0.02658750703131249 -0.010368218245334295 0.003049204563265078
0.02861221292004633 -0.010490977656573346 0.0037980335154632287
0.0253226545444752 -0.004648618149741125 0.004908854265265728
0.03231427862164446 -0.011531361135262676 0.005062871201998836
0.03769786438694536 -0.015008614867559831 0.004868137769647127
0.03783834710035766 -0.011868523115856065 0.005039226183908566
0.04420768294122836 -0.016368406021472007 0.004762487350790826
0.045608716383602546 -0.014321363126986605 0.004770908426507468
0.044630584668055584 -0.010115712414830592 0.005189471889512627
0.055987651238371475 -0.020561094189464522 0.004542470590937085
0.05184563793694662 -0.011406658075259679 0.004940476283919379
0.05601434847499344 -0.013594555616392464 0.005038645374440184
0.05858525125289393 -0.013495617098137326 0.005137027298885504
0.05951981799245984 -0.011602841452831915 0.00540

/tmp/ipykernel_41764/2247328652.py:137: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  gym.logger.warn(


In [110]:
#################################### Testing ###################################

env = Drone()
state = env.reset()
action = np.array([18,0.1])

%matplotlib qt
fig, ax = plt.subplots(1,figsize=(10,5))
traj = []

for t in range(1, 1000):
    
    state, reward, done, _ = env.step(action)
    x = state[0]
    z = -state[1]
    v = state[2]
    theta = -state[3]
    theta_dot = -state[4]
    gamma = state[5]
    traj.append((x,z))

    action = np.array([0,0])
        
    if v<25:
        action[0] = 200
    else:
        action[0] = 0
        
    l = 0.8
    plt.plot([x,x+0.25*l*np.cos(theta)],[z,z+0.25*l*np.sin(theta)],'k-')
    plt.plot([x,x-0.75*l*np.cos(theta)],[z,z-0.75*l*np.sin(theta)],'k-')
    plt.xlim(x-20,x+80)
    plt.ylim(z-25,z+25)

    beam = 50
    angle = 60*np.pi/180
    plt.plot([x,x+beam*np.cos(theta+angle/2)],[z,z+beam*np.sin(theta+angle/2)],'k--')
    plt.plot([x,x+beam*np.cos(theta-angle/2)],[z,z+beam*np.sin(theta-angle/2)],'k--')
    xs = x+beam*np.cos(theta+np.linspace(-angle/2,angle/2,10))
    zs = z+beam*np.sin(theta+np.linspace(-angle/2,angle/2,10))
    plt.plot(xs,zs,'k.',markersize=1)

    x,z = zip(*traj)
    plt.plot(x,z,'b-',markersize=1)
    
    plt.show()
    plt.pause(env.dt)

    
    if done:
        break
    else:
        ax.clear()
print(np.around(state,2), reward, done)
env.close()

0.47646399369458137 -0.013783703119868707 -0.0012813286021767898
57.275390968280306 -0.0069160779625088735 -0.047538920535949165
56.52856112094669 0.05160277512355902 -0.07954907056969043
55.82851017413593 0.10626752042442883 -0.09753230877697365
54.94752351015298 0.1749013735432535 -0.10234437897770678
54.13053989961932 0.2378272311907073 -0.097283868253494
-9.029796158887006 0.24560936332674466 -0.09357261606987197
-8.984876852959248 0.2417759842548937 -0.09273770435114138
-8.870010665710014 0.23238284455022015 -0.09424120910398073
-8.708805641257793 0.21920006255847393 -0.09750571166137766
53.785268702598344 0.324325700141456 -0.08480247246677569
-9.579249588668464 0.2673926930234023 -0.08644408963035521
-9.328526866873156 0.24810162655949392 -0.09077125395064196
-9.30927314366017 0.24471089006745686 -0.0941212071808232
-9.174283685485172 0.23325213661188463 -0.09815246548574652
-9.228612608258123 0.2352879603178088 -0.10063293153248397
-9.060089854183339 0.22149338115967782 -0.1045